Copyright 2021-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Anisotropic tutorial with optimization loop chassis

This notebook walks through four experimental treatments using the optimaztion loop chassis and 2D Univariate Sampler used in the [Anisotropic Metric Tutorial](../docs/examples/anisotropic_tutorial.ipynb). The goal is increase accuracy of learned hyperparameters in order to best recover the response on the held-out test data by training a simple anisotropic `MuyGPS` model on the perturbed training data with smoothness hyperparameter known, while the two `distance scaling` hyperparameters are to be learned. Each iteration of the optimization loop we scale training features using learned length scale hyperparameters, update nearest neighbors lookup using sklearn, update objective function, and run bayes optimization using numpy math backend.

In [ ]:
import sys
for m in sys.modules.keys():
    if m.startswith("Muy"):
        sys.modules.pop(m)
%env MUYGPYS_BACKEND=numpy
%env MUYGPYS_FTYPE=64

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from MuyGPyS._test.sampler import UnivariateSampler2D

from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.deformation import Anisotropy, l2
from MuyGPyS.gp.hyperparameter import AnalyticScale, Parameter
from MuyGPyS.gp.kernels import Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.gp.tensors import make_predict_tensors
from MuyGPyS.optimize.experimental.chassis import optimize_from_tensors_mini_batch
from MuyGPyS.optimize.loss import mse_fn, lool_fn


np.random.seed(0)

points_per_dim = 100 # 60                       # Observations per dimension
train_ratio = 0.1 # 0.05                        # Train/test data split
nugget_noise = HomoscedasticNoise(1e-14)        # Assume no noise in truth
measurement_noise = HomoscedasticNoise(1e-7)    # Noise to perturb train
sim_smoothness = Parameter(1.5)                 # HP smoothness
sim_length_scale0 = Parameter(0.1)              # HP distance scaling dim 0
sim_length_scale1 = Parameter(0.5)              # HP distance scaling dim 1
sampler = UnivariateSampler2D(
    points_per_dim=points_per_dim,
    train_ratio=train_ratio,
    kernel=Matern(
        smoothness=sim_smoothness,
        deformation=Anisotropy(
            l2,
            length_scale0=sim_length_scale0,
            length_scale1=sim_length_scale1,
        ),
    ),
    noise=nugget_noise,
    measurement_noise=measurement_noise,
)
train_features, test_features = sampler.features()
train_responses, test_responses = sampler.sample()

exp_length_scale0 = Parameter("log_sample", (0.01, 1.0))
exp_length_scale1 = Parameter("log_sample", (0.01, 1.0))
muygps = MuyGPS(
    kernel=Matern(
        smoothness=sim_smoothness,
        deformation=Anisotropy(
            l2,
            length_scale0=exp_length_scale0,
            length_scale1=exp_length_scale1,
        ),
    ),
    noise=measurement_noise,
    scale=AnalyticScale(),
)


In [ ]:
opt_kwargs = {
    "loss_fn": lool_fn,
    "verbose": False,
    "random_state": 1,
    "init_points": 5,
    "n_iter": 30,
    "allow_duplicate_points": True,
}

opt_kwargs1b = {
    "loss_fn": lool_fn,
    "verbose": False,
    "random_state": 1,
    "init_points": 25,
    "n_iter": 30,
    "allow_duplicate_points": True,
}

train_table = [
    [
        "treatment",
        "time (s)",
        "train size",
        "batch size", 
        "num epochs",
        "pts probed",
        "opt steps",
        "l.s.0 (p.e.) ",
        "l.s.1 (p.e.) ",
        "scale",
    ]
]

inference_table = [
    [
        "treatment",
        "RMSE",
        "mean diag var",
        "mean c.i. size ",
        "coverage",
    ]
]

inference_data = []

def store_details(
        name,
        batch_size,
        train_size,
        epoch_count,
        muygps_exp,
        nbrs_lookup_final,
        exec_time,
        probe_count,
        opt_steps,
    ):

    # Training results
    learned_sim_length_scale0 = muygps_exp.kernel.deformation.length_scale['length_scale0']()
    pe0 = abs(learned_sim_length_scale0 - sim_length_scale0()) / sim_length_scale0() * 100.0
    learned_sim_length_scale1 = muygps_exp.kernel.deformation.length_scale['length_scale1']()
    pe1 = abs(learned_sim_length_scale1 - sim_length_scale1()) / sim_length_scale1() * 100.0
    ret = [
        name,
        f"{exec_time:.6f}",
        f"{train_size}",
        f"{batch_size}",
        f"{epoch_count}",
        f"{probe_count}",
        f"{opt_steps}",
        f"{learned_sim_length_scale0:.4f} ({pe0:2.0f}%)",
        f"{learned_sim_length_scale1:.4f} ({pe1:2.0f}%)",
        f"{muygps_exp.scale()[0]:.4f}",
    ]
    train_table.append(ret)

    # Inference results
    test_count, _ = test_features.shape
    indices = np.arange(test_count)
    test_nn_indices, _ = nbrs_lookup_final.get_nns(test_features)
    (
        test_crosswise_diffs,
        test_pairwise_diffs,
        test_nn_targets,
    ) = make_predict_tensors(
        indices,
        test_nn_indices,
        test_features,
        train_features,
        train_responses,
    )
    Kcross = muygps_exp.kernel(test_crosswise_diffs)
    Kin = muygps_exp.kernel(test_pairwise_diffs)
    predictions = muygps_exp.posterior_mean(Kin, Kcross, test_nn_targets)
    variances = muygps_exp.posterior_variance(Kin, Kcross)
    confidence_intervals = np.sqrt(variances) * 1.96
    coverage = (
        np.count_nonzero(
            np.abs(test_responses - predictions) < confidence_intervals
        ) / test_count
    )
    ret = [
        name,
        f"{np.sqrt(mse_fn(predictions, test_responses)):.5f}",
        f"{np.mean(variances):.5f}",
        f"{np.mean(confidence_intervals * 2):.5f}",
        f"{coverage:.5f}",
    ]    
    inference_table.append(ret)
    inference_data.append([predictions, confidence_intervals])



Treatment #1a (baseline) - brute force workflow following [Anisotropic Metric Tutorial](../docs/examples/anisotropic_tutorial.ipynb) using single optimization step with a single initial probe point

In [ ]:
(
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
) = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nn_count=30,
    batch_count=sampler.train_count,
    train_count=sampler.train_count,
    num_epochs=1,
    keep_state=False,
    probe_previous=False,
    **opt_kwargs,
)
store_details(
    "single step",
    sampler.train_count,
    sampler.train_count,
    1,
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
)


Treatment #1b - baseline +  configured to exploit and explore same total points 

In [ ]:
(
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
) = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nn_count=30,
    batch_count=sampler.train_count,
    train_count=sampler.train_count,
    num_epochs=1,
    keep_state=False,
    probe_previous=False,
    **opt_kwargs1b,
)
store_details(
    "single step+",
    sampler.train_count,
    sampler.train_count,
    1,
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
)
  

Treatment #2a (opt loop) - each loop interation create new optimizer instance, initialize with new objective function, and do not probe previous points

In [ ]:
(
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
) = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nn_count=30,
    batch_count=(int(sampler.train_count / 5)),
    train_count=sampler.train_count,
    num_epochs=5,
    keep_state=False,
    probe_previous=False,
    **opt_kwargs,
)
store_details(
    "no probe",
    (int(sampler.train_count / 5)),
    sampler.train_count,
    5,
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
)


Treatment #2b (opt loop) - each loop interation create new optimizer instance, initialize with new objective function, and probe previous points

In [ ]:
(
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
) = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nn_count=30,
    batch_count=(int(sampler.train_count / 5)),
    train_count=sampler.train_count,
    num_epochs=5,
    keep_state=False,
    probe_previous=True,
    **opt_kwargs,
)
store_details(
    "probe all",
    (int(sampler.train_count / 5)),
    sampler.train_count,
    5,
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
)


Treatment #2c (opt loop) - each loop interation reuse optimizer instance, set `BayesianOptimization.TargetSpace.target_function` to new objective function, and do not probe previous points 

In [ ]:
(
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
) = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nn_count=30,
    batch_count=sampler.train_count,
    train_count=sampler.train_count,
    num_epochs=5,
    keep_state=True,
    probe_previous=False,
    **opt_kwargs,
)
store_details(
    "reuse opt",
    sampler.train_count,
    sampler.train_count,
    5,
    muygps_optloop,
    nbrs_lookup_final,
    exec_time,
    probe_count,
    opt_steps,
)


Training and inference results

In [ ]:
# Print tables
max_len = max(len(header) for header in train_table[0])
sep = "-" * max_len
for row in train_table:
    print("|".join(header.ljust(max_len) for header in row))
    if row == train_table[0]:
        print("|".join(sep for _ in row))
print("\n")
max_len = max(len(header) for header in inference_table[0])
sep = "-" * max_len
for row in inference_table:
    print("|".join(header.ljust(max_len) for header in row))
    if row == inference_table[0]:
        print("|".join(sep for _ in row))

# Load results
results = []
for predictions, confidence_intervals in inference_data:
    (
        resl_im,
        conf_im,
        covr_im,
        resl_mag,
        conf_mag,
        covr_mag,
    ) = sampler._get_images(predictions, confidence_intervals)
    results.append(
        [resl_im, conf_im, covr_im, resl_mag, conf_mag, covr_mag,]
    )

# Plot residuals
im = []
fig, axes = plt.subplots(1, 5, figsize=(50, 7))
for index, result in enumerate(results):
    resl_im, _, _, resl_mag, _, _ = result
    im.append(
        axes[index].imshow(
            resl_im,
            vmin=-resl_mag,
            vmax=resl_mag,
            cmap="coolwarm"
        )
    )
    axes[index].set_title(train_table[index + 1][0], fontsize=20)
fig.colorbar(im[0], ax=axes.ravel().tolist())
fig.suptitle("Residual", x=0.45, fontsize=26)
axes[0].set_xlabel("Axis 0", fontsize=20)
axes[0].set_ylabel("Axis 1", fontsize=20)
plt.show()
# Plot CI Magnitude
im = []
fig, axes = plt.subplots(1, 5, figsize=(50, 7))
for index, result in enumerate(results):
    _, conf_im, _, _, conf_mag, _ = result
    im.append(
        axes[index].imshow(
            conf_im, vmin=0.0, vmax=conf_mag, cmap="inferno"
        )
    )
    axes[index].set_title(train_table[index + 1][0], fontsize=20)
fig.colorbar(im[0], ax=axes.ravel().tolist())
fig.suptitle("CI Magnitude", x=0.45, fontsize=26)
axes[0].set_xlabel("Axis 0", fontsize=20)
axes[0].set_ylabel("Axis 1", fontsize=20)
plt.show()
# Plot |Residual| - CI
im = []
fig, axes = plt.subplots(1, 5, figsize=(50, 7))
for index, result in enumerate(results):
    _, _, covr_im, _, _, covr_mag = result
    im.append(
        axes[index].imshow(
            covr_im, vmin=-covr_mag, vmax=covr_mag, cmap="coolwarm"
        )
    )
    axes[index].set_title(train_table[index + 1][0], fontsize=20)
fig.colorbar(im[0], ax=axes.ravel().tolist())
fig.suptitle("|Residual| - CI", fontsize=26)
axes[0].set_xlabel("Axis 0", fontsize=20)
axes[0].set_ylabel("Axis 1", fontsize=20)
# plt.tight_layout()
plt.show()
